In [84]:
import matplotlib.pyplot as plt
import cv2

image = '../traf.bmp'

#Load image
#I = plt.imread(image)
I=cv2.imread(image)
I=I.astype("float64")
Green,Blue,Red=cv2.split(I)
#plt.figure(figsize=(10,10))
#plt.imshow(I,cmap='gray')

#Check data type of image
print('Data type: '+str(Blue.dtype))
print('Pixel intensity range: (%d,%d)'%(I.min(),I.max()))

#Print image shape
print(I.shape)


Data type: float64
Pixel intensity range: (0,255)
(1080, 1920, 3)


In [85]:

from scipy.fftpack import dct
from scipy.fftpack import idct
import numpy as np

def dct2(f):
    """2D Discrete Cosine Transform

    Args:
        f: Square array

    Returns: 
        2D DCT of f
    """
    return dct(dct(f, axis=0, norm='ortho' ),axis=1, norm='ortho')




def idct2(f):
    """2D Inverse Discrete Cosine Transform

    Args:
        f: Square array

    Returns: 
        2D Inverse DCT of f
    """
    return idct(idct(f, axis=0 , norm='ortho'), axis=1 , norm='ortho')


In [86]:
 def fun(I):   
    patch_size = 8  #8x8 patch
    I_dct = np.zeros_like(I)
    for i in range(0,I.shape[0],patch_size):
        for j in range(0,I.shape[1],patch_size):
            I_dct[i:(i+patch_size),j:(j+patch_size)] = dct2(I[i:(i+patch_size),j:(j+patch_size)])
    thresh = 0.04
    I_thresh = I_dct * (np.absolute(I_dct) > thresh*np.max(np.absolute(I_dct)))
    return  I_thresh


In [87]:
dct_b=fun(Blue)
dct_g=fun(Green)
dct_r=fun(Red)

In [88]:
def revers(I_thresh):
    patch_size = 8  #8x8 patch
    I_comp = np.zeros(I_thresh.shape,dtype="float64")
    for i in range(0,I.shape[0],patch_size):
        for j in range(0,I.shape[1],patch_size):
            I_comp[i:(i+patch_size),j:(j+patch_size)] = idct2(I_thresh[i:(i+patch_size),j:(j+patch_size)])
    return  I_comp

In [89]:
idct_b=revers(dct_b)
idct_g=revers(dct_g)
idct_r=revers(dct_r)



In [90]:
I_comp=cv2.merge([idct_b,idct_g,idct_r])


In [91]:

#Plot decompressed (reconstructed image)
#plt.figure(figsize=(30,10))
#plt.imshow(I_comp)



In [92]:

#cv2.imwrite("dct0.02.min.png",I_comp)

In [93]:

#Compute Peak Signal to Noise Ratio (PSNR)
MSE = np.sum((I-I_comp)**2)/I.size
PSNR = 10*np.log10(np.max(I)**2/MSE)
print('PSNR: %.4f dB'%PSNR)


PSNR: 24.0376 dB


In [94]:
def tobin(img):
    im_gray =cv2.cvtColor(img.astype("uint8"),cv2.COLOR_BGR2GRAY)
   
    (thresh, im_bw) = cv2.threshold(im_gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    cv2.imwrite('bw_img.png', im_bw)

In [83]:
tobin(I_comp)